In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from catboost import CatBoostClassifier

In [4]:
xtrain = pd.read_csv("../data/X_train.csv")
xtrain = xtrain.iloc[:, 1:]

xtest = pd.read_csv("../data/X_test.csv")
xtest = xtest.iloc[:, 1:]

ytrain = pd.read_csv("../data/Y_train.csv")
ytrain = ytrain.drop('id', axis = 1)

Slow

In [17]:
steps = [("scaler", StandardScaler()), ("classifier", SVC(max_iter=-1, probability=False, random_state=None, shrinking=True))]

pipeline = Pipeline(steps = steps)

parameters = {"classifier__kernel": ["rbf"],
              "classifier__gamma": [0.00045, 0.00055, 0.0005],
              "classifier__tol": [0.0085, 0.009, 0.008],
              "classifier__C": [0., 0.625, 0.65],
              "classifier__class_weight": ["balanced"]
             }


grid = GridSearchCV(pipeline, parameters, verbose=5, n_jobs=-1, cv = 5, scoring = 'balanced_accuracy')

grid.fit(xtrain.values, ytrain.values.ravel())

print(grid.best_score_)
print(grid.best_params_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 24.7min finished
0.7076851851851852
{'classifier__C': 0.65, 'classifier__class_weight': 'balanced', 'classifier__gamma': 0.00045, 'classifier__kernel': 'rbf', 'classifier__tol': 0.0085}


Slower

In [7]:
model = CatBoostClassifier(loss_function="MultiClassOneVsAll", auto_class_weights="Balanced", task_type="GPU")

grid = {'learning_rate': [0.01, 0.05, 0.075],
        'depth': [6],
        'l2_leaf_reg': [3, 4]}

grid_search_result = model.grid_search(grid,
                                       cv=5,
                                       X=xtrain.values, 
                                       y=ytrain.values.ravel(), 
                                       verbose=True,
                                       plot=False)

print(grid_search_result['params'])

bestTest = 0.4305065112
bestIteration = 980
0:	loss: 0.4305065	best: 0.4305065 (0)	total: 57.2s	remaining: 4m 45s
bestTest = 0.4291310929
bestIteration = 168
1:	loss: 0.4291311	best: 0.4291311 (1)	total: 1m 51s	remaining: 3m 42s
bestTest = 0.4359025751
bestIteration = 151
2:	loss: 0.4359026	best: 0.4291311 (1)	total: 2m 45s	remaining: 2m 45s
bestTest = 0.4284243364
bestIteration = 979
3:	loss: 0.4284243	best: 0.4284243 (3)	total: 3m 46s	remaining: 1m 53s
bestTest = 0.4272166189
bestIteration = 169
4:	loss: 0.4272166	best: 0.4272166 (4)	total: 4m 45s	remaining: 57s
bestTest = 0.4330224387
bestIteration = 107
5:	loss: 0.4330224	best: 0.4272166 (4)	total: 5m 41s	remaining: 0us
Estimating final quality...
{'depth': 6, 'l2_leaf_reg': 4, 'learning_rate': 0.05}


In [10]:
svc_ = SVC(class_weight='balanced', gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.0085, verbose=False, C=1.1666666666666665)

svc_.fit(xtrain.values, ytrain.values.ravel())
predictions = svc_.predict(xtest.values)

filename = "svc"

test_data =  pd.read_csv("../data/x_test.csv")
test_data["y"] = predictions
test_data[["id", "y"]].to_csv("../results/"+filename+".csv", index = False)

In [9]:
catboost_ = CatBoostClassifier(loss_function="MultiClassOneVsAll", auto_class_weights="Balanced", task_type="GPU",
    learning_rate=0.05, depth=6, l2_leaf_reg=, verbose=False)
    
catboost_.fit(xtrain.values, ytrain.values.ravel())
predictions = catboost_.predict(xtest.values)

filename = "catboost"

test_data =  pd.read_csv("../data/x_test.csv")
test_data["y"] = predictions
test_data[["id", "y"]].to_csv("../results/"+filename+".csv", index = False)

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.svm import SVC

nr_features = 1000

# read the data from the training set
X_given = pd.read_csv('../data/X_train.csv', header=0, usecols=range(1, nr_features)).to_numpy()

# read the labels of the training set
y_given = pd.read_csv('../data/y_train.csv', header=0, usecols=[1])
y_given = y_given.to_numpy().ravel()

model = SVC(C=1.1666666666666665, gamma='auto', class_weight='balanced')
model.fit(X_given, y_given)

# read the test data
X_test = pd.read_csv('../data/X_test.csv', header=0, usecols=range(1, nr_features))

# predict the labels for the test set
y_predict = model.predict(X_test)

# write result to csv file
result = pd.DataFrame(data=y_predict)
result.index.name = 'id'
result.columns = ['y']
result.to_csv('result.csv')
